In [7]:
%matplotlib
import torch
from torch import nn
from d2l import torch as d2l

Using matplotlib backend: TkAgg


In [8]:
class Data(d2l.DataModule):
    def __init__(self, num_train, num_val, num_inputs, batch_size):
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, num_inputs)
        noise = torch.randn(n,1) * 0.01
        w, b = torch.ones((num_inputs, 1)) * 0.01, 0.05
        self.y = torch.matmul(self.X, w) + b + noise

    def get_dataloader(self, train):
        """
        这个函数的作用是获取数据加载器。
        
        它根据是训练集还是验证集来选择相应的数据切片,
        然后使用 get_tensorloader 方法创建并返回一个 TensorLoader 对象。
        
        参数:
        train (bool): 如果为 True,返回训练数据加载器;如果为 False,返回验证数据加载器。
        
        返回:
        TensorLoader: 包含选定数据的数据加载器。
        """
        # slice函数用于创建一个切片对象,用来选择数据的子集
        # 如果是训练集(train=True),选择从索引0到num_train的数据
        # 如果是验证集(train=False),选择从num_train到末尾的数据
        # 确保 train 是布尔值
    def get_dataloader(self, train):
        i = slice(0, self.num_train) if train else slice(self.num_train, None)
        return self.get_tensorloader([self.X, self.y], train, i)

In [9]:
def l2_penalty(w):
    return (w ** 2).sum() / 2


In [10]:
class WeightDecayScratch(d2l.LinearRegressionScratch):
    def __init__(self, num_inputs, lambd, lr, sigma=0.01):
        super().__init__(num_inputs, lr, sigma)
        self.save_hyperparameters()

    def loss(self, y_hat, y):
        # 第一项是均方误差项，第二项是权重衰减项
        return (super().loss(y_hat, y) +
                self.lambd * l2_penalty(self.w))


In [11]:
data = Data(num_train=20, num_val=100, num_inputs=200, batch_size=5)
trainer = d2l.Trainer(max_epochs=10)

def train_scratch(lambd):
    model = WeightDecayScratch(num_inputs=200, lambd=lambd, lr=0.01)
    model.board.yscale='log'
    trainer.fit(model, data)
    print('L2 norm of w:', float(l2_penalty(model.w)))


In [13]:
train_scratch(0)

L2 norm of w: 0.010814397595822811


In [14]:
train_scratch(3)

L2 norm of w: 0.0016116609331220388


#### Concise Implementation

In [15]:
class WeightDecay(d2l.LinearRegression):
    def __init__(self, wd, lr):
        super().__init__(lr)
        # save_hyperparameters 的作用是将模型的超参数保存下来，以便后续使用和记录。
        # 它通常会保存 __init__ 方法中定义的参数，如 lr 和 wd。
        # 这对于实验追踪、模型复现和结果分析非常有用。
        # 保存的超参数可以在训练过程中被访问，也可以在模型保存时一同保存下来。
        self.save_hyperparameters()
        self.wd = wd
    
    
    def configure_optimizers(self):
        """
        这个函数通常在模型训练开始前被调用，用于配置优化器。
        
        具体来说，它会在以下情况下被调用：
        1. 当使用PyTorch Lightning等高级训练框架时，在训练循环开始前自动调用。
        2. 在手动设置训练流程时，通常在创建模型实例后、开始训练循环前显式调用。
        3. 如果需要中途更改优化器设置，可能会在训练过程中再次调用。

        函数返回配置好的优化器，用于后续的参数更新过程。
        """
        # self.net.weight 是神经网络层的权重参数
        # self.net.bias 是神经网络层的偏置参数
        # 我们对权重应用权重衰减(weight_decay),但不对偏置应用
        return torch.optim.SGD([
            {'params': self.net.weight, 'weight_decay': self.wd},
            {'params': self.net.bias}
        ], lr=self.lr)


In [18]:
model = WeightDecay(wd=3, lr=0.01)
model.board.yscale = 'log'
trainer.fit(model, data)

# model.get_w_b() 是一个方法，用于获取模型的权重和偏置参数
# 它返回一个元组，其中第一个元素是权重，第二个元素是偏置
# 我们使用 [0] 来获取权重部分

weight = model.get_w_b()[0]
print('权重的L2范数:', float(l2_penalty(weight)))


L2 norm of w: 0.014405453577637672
